In [305]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import os, re, locale

In [315]:
player_name = 'Lionel Messi'
with open(f'players_history_raw/{player_name}_raw.txt', encoding='utf-8') as f:
    soup = bs(f.read())

transfer_history = soup.find('div', {'class' : 'box'})
transfer_history

<div class="box viewport-tracking" data-viewport="Transferhistorie">
<h2 class="content-box-headline">
        Transfer history     </h2>
<div class="grid tm-player-transfer-history-grid tm-player-transfer-history-grid tm-player-transfer-history-grid--heading">
<div class="grid__heading grid__heading--center tm-player-transfer-history-grid__season">Season</div>
<div class="grid__heading grid__heading--center tm-player-transfer-history-grid__date">Date</div>
<div class="grid__heading tm-player-transfer-history-grid__old-club">Left</div>
<div class="grid__heading tm-player-transfer-history-grid__new-club">Joined</div>
<div class="grid__heading grid__heading--ar tm-player-transfer-history-grid__market-value">MV</div>
<div class="grid__heading grid__heading--ar tm-player-transfer-history-grid__fee">Fee</div>
<span class="grid__heading tm-player-transfer-history-grid__link--placeholder"></span>
</div>
<div class="grid tm-player-transfer-history-grid">
<div class="grid__cell grid__cell--cent

In [232]:
def get_transfer_history(player_name: str) -> pd.DataFrame:
    """
        Reads in the raw player files and parses their transfer history.
    """
    columns = {
        'season': 'tm-player-transfer-history-grid__season',
        'date': 'tm-player-transfer-history-grid__date',
        'old_team': 'tm-player-transfer-history-grid__old-club',
        'new_team': 'tm-player-transfer-history-grid__new-club',
        'market_value': 'tm-player-transfer-history-grid__market-value',
        'transfer_fee': 'tm-player-transfer-history-grid__fee'
    }
    
    with open(f'players_history_raw/{player_name}_raw.txt', encoding='utf-8') as f:
        soup = bs(f.read())
        
    transfer_history = soup.find('div', {'class' : 'box'})
    t_table = []
    for i, (k, v) in enumerate(columns.items()):
        t_items = []
        for j in transfer_history.find_all('div', {'class': v}):
            t_items.append(j.text.strip())
        t_table.append(t_items)
    current_league = soup.find('span', {'class': 'data-header__league'})
    current_league = current_league.text.strip() if not pd.isnull(current_league) else np.nan
        
    df = pd.DataFrame(t_table).T[1:]
    df.columns = columns.keys()
    df = clean_transfer_history(df)
    df['name'] = player_name
    df['current_league'] = current_league
    
    if not pd.isnull(current_league):
        df['current_league_country'] = soup.find('div', {'class': 'data-header__club-info'}).find(
            'span', {'class': 'data-header__content'}).find('img')['alt']
    else:
        df['current_league_country'] = np.nan
    
    df = df[['name', 'from_date', 'to_date', 'old_team', 'new_team', 'market_value', 'transfer_fee',
             'current_league', 'current_league_country']]
    
    return df

def clean_transfer_history(df: pd.DataFrame) -> pd.DataFrame:
    """
        Cleans the transfer history and only keeps relevant values.
    """
    df.dropna(inplace=True)
    df = pd.DataFrame([[np.nan] * len(df.columns)], columns=df.columns).append(df, ignore_index=True)
    df['from_date'] = df['date'].shift(-1)
    df.rename(columns={'date': 'to_date'}, inplace=True)
    df.drop(columns=['season'], inplace=True)
    df['old_team'][0] = df['new_team'][1]
    df.replace('-', np.nan, inplace=True)
    df.replace('free transfer', 0, inplace=True)
    for c in ['market_value', 'transfer_fee']:
        df[c] = df[c].replace('[\€,]', '', regex=True)
        nums = []
        for i, r in df.iterrows():
            if not pd.isnull(r[c]) and 'loan' not in str(r[c]).lower() and '?' not in str(r[c]):
                try:
                    num = float("".join(re.findall("[\d.]+", str(x))))
                except:
                    num = np.nan
                unit = "".join(re.findall("[a-zA-Z]+", str(r[c])))
                if unit == "m":
                    num *= 1000000
                elif unit == "k":
                    num *= 1000
                nums.append(num)
            else:
                nums.append(np.nan)
        df[c] = nums
    
    return df

def clean_countries_players(csv: str) -> pd.DataFrame:
    """
        Reads the csv files for each country and cleans the file.
    """
    df = pd.read_csv(f'countries_players/{csv}')
    df.drop(columns=['Unnamed: 0', 'url'], inplace=True)
    df.rename(columns={'tm-shirt-number': 'shirt_number', 'current-team': 'current_team', 
                       'market-value': 'current_market_value'}, inplace=True)
    
    return df

In [233]:
for country_csv in os.listdir('countries_players/'):
    country_df = clean_countries_players(country_csv)
    dfs = []
    for i, r in country_df.iterrows():
        dfs.append(country_df.merge(get_transfer_history(r['name']), how='inner', on='name'))
    master_df = pd.concat(dfs).reset_index(drop=True)
    if not os.path.exists('country_player_history/'):
        os.makedirs('country_player_history/')
    master_df.to_csv(f"country_player_history/{country_csv.split('_')[0]}.csv", index=False, header=True)

In [257]:
country_player_df = pd.DataFrame(columns=['id', 'current_league_country'])
for country_player_history_csv in os.listdir('country_player_history/'):
    ph_df = pd.read_csv(f'country_player_history/{country_player_history_csv}')[[
        'name', 'current_league_country']].drop_duplicates().rename(columns={'name': 'id'})
    country_player_df = pd.concat([country_player_df, ph_df])
country_player_df.reset_index(drop=True, inplace=True)
country_player_df['current_league_country'] = country_player_df['current_league_country'].str.replace(
    'Korea, South', 'South Korea')

In [280]:
country_team_df = pd.DataFrame(columns=['old_team', 'current_league_country'])
for country_team_history_csv in os.listdir('country_player_history/'):
    th_df = pd.read_csv(f'country_player_history/{country_team_history_csv}')[[
        'current_team', 'current_league_country']].drop_duplicates()
    country_team_df = pd.concat([country_team_df, th_df])
country_team_df.reset_index(drop=True, inplace=True)
country_team_df.drop_duplicates(inplace=True)
country_team_df['current_league_country'] = country_team_df['current_league_country'].str.replace(
    'Korea, South', 'South Korea')

In [283]:
country_player_df.to_csv('player_current_league_country.csv', index=False, header=True)
country_team_df.to_csv('team_league.csv', index=False, header=True)

In [287]:
get_transfer_history('Kylian Mbappé')

,name,from_date,to_date,old_team,new_team,market_value,transfer_fee,current_league,current_league_country
0,Kylian Mbappé,"Jul 1, 2018",NaN,Paris SG,NaN,NaN,NaN,Ligue 1,France
1,Kylian Mbappé,"Jun 30, 2018","Jul 1, 2018",Monaco,Paris SG,NaN,NaN,Ligue 1,France
2,Kylian Mbappé,"Aug 31, 2017","Jun 30, 2018",Paris SG,Monaco,NaN,NaN,Ligue 1,France
3,Kylian Mbappé,"Dec 1, 2015","Aug 31, 2017",Monaco,Paris SG,NaN,NaN,Ligue 1,France
4,Kylian Mbappé,"Jul 1, 2015","Dec 1, 2015",Monaco U19,Monaco,NaN,NaN,Ligue 1,France
5,Kylian Mbappé,"Jul 1, 2013","Jul 1, 2015",Monaco Youth,Monaco U19,NaN,NaN,Ligue 1,France
6,Kylian Mbappé,"Jul 1, 2011","Jul 1, 2013",Clairef. Yth.,Monaco Youth,NaN,NaN,Ligue 1,France
7,Kylian Mbappé,NaN,"Jul 1, 2011",AS Bondy Youth,Clairef. Yth.,NaN,NaN,Ligue 1,France


In [372]:
for team in ['Real Madrid']:
    money_in, money_out = 0, 0
    for country_team_history_csv in os.listdir('country_player_history/'):
        th_df = pd.read_csv(f'country_player_history/{country_team_history_csv}')
        money_in += np.nansum(th_df[th_df['old_team'] == team]['transfer_fee'].tolist())
        money_out += np.nansum(th_df[th_df['new_team'] == team]['transfer_fee'].tolist())
    print(f'{team}\nMoney in: {"${:0,.0f}".format(money_in)}\nMoney out: {"${:0,.0f}".format(money_out)}\n')

Real Madrid
Money in: $595,950,000
Money out: $845,400,000



In [373]:
teams = ['Chelsea', 'Real Madrid']
money_in, money_out = 0, 0
for country_team_history_csv in os.listdir('country_player_history/'):
    df = pd.read_csv(f'country_player_history/{country_team_history_csv}')
    money_in += np.nansum(df[(df['old_team'] == teams[0]) & (df['new_team'] == teams[1])]['transfer_fee'].tolist())
    if len(df[(df['old_team'] == teams[0]) & (df['new_team'] == teams[1])][['name', 'transfer_fee']]) > 0:
        print(df[(df['old_team'] == teams[0]) & (df['new_team'] == teams[1])][['name', 'transfer_fee']])
    money_out += np.nansum(df[(df['old_team'] == teams[1]) & (df['new_team'] == teams[0])]['transfer_fee'].tolist())
    if len(df[(df['old_team'] == teams[1]) & (df['new_team'] == teams[0])][['name', 'transfer_fee']]) > 0:
        print(df[(df['old_team'] == teams[1]) & (df['new_team'] == teams[0])][['name', 'transfer_fee']])
print(f'\nFrom {teams[1]} to {teams[0]}: {"${:0,.0f}".format(money_in)}\nFrom {teams[0]} to {teams[1]}: {"${:0,.0f}".format(money_out)}\n')

                 name  transfer_fee
115  Thibaut Courtois    35000000.0
              name  transfer_fee
213  Mateo Kovacic           NaN
              name  transfer_fee
212  Mateo Kovacic    45000000.0
214  Mateo Kovacic           NaN
              name  transfer_fee
6  Antonio Rüdiger           0.0
             name  transfer_fee
76  Álvaro Morata    66000000.0

From Real Madrid to Chelsea: $35,000,000
From Chelsea to Real Madrid: $111,000,000



In [380]:
team = 'Benfica'
money_in = 0
for country_team_history_csv in os.listdir('country_player_history/'):
    df = pd.read_csv(f'country_player_history/{country_team_history_csv}')
    money_in += np.nansum(df[(df['old_team'] == team)]['transfer_fee'].tolist())
    if len(df[(df['old_team'] == team)][['name', 'transfer_fee']]) > 0:
        print(df[(df['old_team'] == team)][['name', 'transfer_fee']])
print(f'\nMoney in for {team}: {"${:0,.0f}".format(money_in)}')

                 name  transfer_fee
61     Enzo Fernández           NaN
124  Nicolás Otamendi           NaN
185    Ángel Di María    33000000.0
               name  transfer_fee
24   Jan Vertonghen           0.0
172     Axel Witsel    40000000.0
      name  transfer_fee
1  Ederson    40000000.0
               name  transfer_fee
146  Steven Vitória           0.0
148  Steven Vitória           NaN
             name  transfer_fee
32  Alexander Bah           NaN
57    Daniel Wass     1200000.0
59    Daniel Wass           NaN
                  name  transfer_fee
56        Raúl Jiménez    38000000.0
58        Raúl Jiménez           NaN
64  Rogelio Funes Mori     3600000.0
66  Rogelio Funes Mori           NaN
               name  transfer_fee
14       Rúben Dias    71600000.0
79   Bernardo Silva    15750000.0
81   Bernardo Silva           NaN
87       João Mário           NaN
153   António Silva           NaN
171    João Cancelo    15000000.0
173    João Cancelo           NaN
232      João Fél